In [5]:
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model

def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)
    
false = False
true = True
null = None

In [6]:
print "Reading data..."
businesses = list(parseData("business.json"))
reviews = list(parseData("review.json"))
users = list(parseData("user.json"))
print "done"

Reading data...
done


In [21]:
user_visited_list = defaultdict(list)
user_averages = defaultdict(float)
business_visited_list = defaultdict(list)
business_info = defaultdict(dict)

for r in reviews:
    user_visited_list[r['user_id']].append((r['business_id'], r['stars']))
    business_visited_list[r['business_id']].append(r['user_id'])

for u in users:
    user_averages[u['user_id']] = u['average_stars']
    
for b in businesses:
    business_info[b['business_id']] = b

In [8]:
def PearsonSim(user1, user2):
    user1_set = set([b[0] for b in user_visited_list[user1]])
    user2_set = set([b[0] for b in user_visited_list[user2]])
    same_visits = user1_set.intersection(user2_set)
    
    user1_dict = defaultdict(int)
    user2_dict = defaultdict(int)
    for b in user_visited_list[user1]:
        user1_dict[b[0]] = b[1]
    for b in user_visited_list[user2]:
        user2_dict[b[0]] = b[1]
    
    top = 0.0
    bottom = 0.0
    user1_avg = user_averages[user1]
    user2_avg = user_averages[user2]
    user1_bot = 0.0
    user2_bot = 0.0
    
    for b in same_visits:
        user1_score = user1_dict[b] - user1_avg
        user2_score = user2_dict[b] - user2_avg
        
        top += user1_score * user2_score
        user1_bot += user1_score**2
        user2_bot += user2_score**2
        
    bottom += (user1_bot * user2_bot)**(0.5)
    if bottom == 0:
        return 0
    
    return (top * 1.0 / bottom)

In [ ]:
for r in reviews:
    if len(business_visited_list[r['business_id']]) > 2:
        score = PearsonSim(business_visited_list[r['business_id']][1], business_visited_list[r['business_id']][0])
        if score > -1.0 and score < 1.0 and score != 0:
            print score

In [15]:
def recommender(user):
    visited = [b[0] for b in user_visited_list[user]]
    businesses = defaultdict(float)
    return_list = []
    
    for v in visited:
        for u in business_visited_list[v]:
            if u != user:
                score = PearsonSim(u, user)
                for b in [x[0] for x in user_visited_list[u]]:
                    if b not in visited:
                        businesses[b] += score
                        
    for b, s in businesses.iteritems():
        return_list.append((b, s))
        
    return_list.sort(key=lambda x: x[1])
    return_list.reverse()
    
    return return_list

In [18]:
visited_test = [b[0] for b in user_visited_list[users[0]['user_id']]]
test_r = recommender(users[0]['user_id'])

In [23]:
for b in visited_test:
    print business_info[b]['name']

CUT by Wolfgang Puck
Chart House
Vic & Anthony's Steakhouse
Hennessey's Tavern
Top of Binion's Steakhouse
Grotto Ristorante
Las Vegas Bull
Pizza Rock
Rao's
Enlightened Soul Tattoo & Piercing
Joe's Seafood Prime Steak & Stone Crab
Fleur by Hubert Keller
Redwood Bar and Grill
D Grill


In [25]:
for b in test_r[:10]:
    print business_info[b[0]]['name']

Mon Ami Gabi
Bacchanal Buffet
Oyster Bar
Monta Ramen
Rollin Smoke Barbeque
Shake Shack
Earl of Sandwich
Gordon Ramsay BurGR
Island Flavor
JINYA Ramen Bar
